In [2]:

import numpy as np
import tensorflow as tf
import sklearn
import pandas as pd
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Read data

In [3]:
item_df = pd.read_csv("data/items.csv",encoding='utf-8', delimiter="\t")
search_df = pd.read_csv("data/search_15k6.csv",encoding='utf-8', delimiter="\t")

In [4]:
item_df

,id,name,avg_rating,n_reviews,n_sold,price,n_loved,n_rate_5,rate_4,rate_3,rate_2,rate_1,rate_with_cmt,rate_with_imgvid,shop_name,shop_n_review,shop_n_product,shop_rate_feedback,shop_time_feedback,shop_age,shop_follower
0,0,Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - B...,4.952381,42,7800.0,1299000.0,3733,40,2,0,0,0,39,32,thinhphat_mobile,2331,259,79,6397,9.449176e+07,25619
1,1,điện thoại Samsung Galaxy S7 chính hãng / full...,4.909091,33,6600.0,1899000.0,3063,32,0,0,1,0,31,29,thinhphat_mobile,2331,259,79,6397,9.449176e+07,25620
2,2,Điện thoại Nokia 1280 Nguyên zin Mới tinh chín...,4.517241,58,8800.0,189900.0,3607,49,3,1,1,5,41,37,annhien_mobile,1444,499,89,5244,9.447916e+07,27558
3,3,ĐIỆN THOẠI NOKIA 1202 MỚI TINH FULL SẠC PIN CH...,4.696970,33,6100.0,175000.0,3111,30,0,1,0,2,25,18,thinhphat_mobile,2331,259,79,6397,9.449176e+07,25620
4,4,điện thoại Samsung Galaxy S7 Edge 2 sim ram 4G...,5.000000,17,5400.0,2799000.0,272,17,0,0,0,0,14,12,smartphonehcm,802,327,72,8502,7.995708e+06,1547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15633,15643,Card âm thanh 7.1ch kênh gắn ngoài kết nối USB...,4.428571,7,50.0,28000.0,10,1,0,0,0,1,2,1,kebidutechnology.vn,5139,457,18,None,7.496714e+07,1983
15634,15644,Loa Vi Tính Sada V-117 Loa Máy Tính USB Có Dây...,4.800000,10,30.0,280000.0,21,8,2,0,0,0,3,1,nhungdo298,588,170,87,3857,1.248920e+08,1903
15635,15645,Usb Bluetooth HJX-001 tạo bluetooth cho loa & ...,4.928571,14,33.0,50000.0,10,1,0,0,0,0,0,0,hunglongpc,15431,1064,94,4498,9.661716e+07,4226
15636,15646,Loa 2.0 Kisonli V310,4.333333,15,41.0,34000.0,8,12,0,1,0,2,5,3,meocon021184,1257,1634,87,10118,1.401270e+08,986


In [5]:
search_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15648 entries, 0 to 15647
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            15648 non-null  int64  
 1   name          15648 non-null  object 
 2   category      15648 non-null  object 
 3   n_sold        15648 non-null  float64
 4   price         15648 non-null  float64
 5   shop_address  15648 non-null  object 
 6   image_url     15648 non-null  object 
 7   url           15648 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 978.1+ KB


In [6]:
#Convert shop_rate_feedback and shop_time_feedback's None values to zeros

item_df[['shop_rate_feedback']] = item_df[['shop_rate_feedback']].replace("None", 0).astype(int)
item_df[['shop_time_feedback']] = item_df[['shop_time_feedback']].replace("None", 0).astype(int)

In [7]:
full_df = search_df.merge(item_df, how='inner', on=['id'], suffixes=(None,'_2'))

In [8]:
full_df

,id,name,category,n_sold,price,shop_address,image_url,url,name_2,avg_rating,n_reviews,n_sold_2,price_2,n_loved,n_rate_5,rate_4,rate_3,rate_2,rate_1,rate_with_cmt,rate_with_imgvid,shop_name,shop_n_review,shop_n_product,shop_rate_feedback,shop_time_feedback,shop_age,shop_follower
0,0,Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - B...,DIENTHOAI,7800.0,1299000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/654146756efe4b5fe7cf...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...,Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - B...,4.952381,42,7800.0,1299000.0,3733,40,2,0,0,0,39,32,thinhphat_mobile,2331,259,79,6397,9.449176e+07,25619
1,1,điện thoại Samsung Galaxy S7 chính hãng / full...,DIENTHOAI,6600.0,1899000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/622be579c39efa0ea92b...,https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%...,điện thoại Samsung Galaxy S7 chính hãng / full...,4.909091,33,6600.0,1899000.0,3063,32,0,0,1,0,31,29,thinhphat_mobile,2331,259,79,6397,9.449176e+07,25620
2,2,Điện thoại Nokia 1280 Nguyên zin Mới tinh chín...,DIENTHOAI,8800.0,189900.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/c92a82f0a1e8076fcc1e...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...,Điện thoại Nokia 1280 Nguyên zin Mới tinh chín...,4.517241,58,8800.0,189900.0,3607,49,3,1,1,5,41,37,annhien_mobile,1444,499,89,5244,9.447916e+07,27558
3,3,ĐIỆN THOẠI NOKIA 1202 MỚI TINH FULL SẠC PIN CH...,DIENTHOAI,6100.0,175000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/b36a13ed46c0e3feab04...,https://shopee.vn/%C4%90I%E1%BB%86N-THO%E1%BA%...,ĐIỆN THOẠI NOKIA 1202 MỚI TINH FULL SẠC PIN CH...,4.696970,33,6100.0,175000.0,3111,30,0,1,0,2,25,18,thinhphat_mobile,2331,259,79,6397,9.449176e+07,25620
4,4,điện thoại Samsung Galaxy S7 Edge 2 sim ram 4G...,DIENTHOAI,5400.0,2799000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/deb114c783e8feb58d6c...,https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%...,điện thoại Samsung Galaxy S7 Edge 2 sim ram 4G...,5.000000,17,5400.0,2799000.0,272,17,0,0,0,0,14,12,smartphonehcm,802,327,72,8502,7.995708e+06,1547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15633,15643,Card âm thanh 7.1ch kênh gắn ngoài kết nối USB...,AUDIO,50.0,28000.0,Nước ngoài,https://cf.shopee.vn/file/da800af7ffa1176f210d...,https://shopee.vn/Card-%C3%A2m-thanh-7.1ch-k%C...,Card âm thanh 7.1ch kênh gắn ngoài kết nối USB...,4.428571,7,50.0,28000.0,10,1,0,0,0,1,2,1,kebidutechnology.vn,5139,457,18,0,7.496714e+07,1983
15634,15644,Loa Vi Tính Sada V-117 Loa Máy Tính USB Có Dây...,AUDIO,30.0,280000.0,Hà Nội,https://cf.shopee.vn/file/ed49a3f85a2b24267800...,https://shopee.vn/Loa-Vi-T%C3%ADnh-Sada-V-117-...,Loa Vi Tính Sada V-117 Loa Máy Tính USB Có Dây...,4.800000,10,30.0,280000.0,21,8,2,0,0,0,3,1,nhungdo298,588,170,87,3857,1.248920e+08,1903
15635,15645,Usb Bluetooth HJX-001 tạo bluetooth cho loa & ...,AUDIO,33.0,50000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/831a33c5745fc36ad0e3...,https://shopee.vn/Usb-Bluetooth-HJX-001-t%E1%B...,Usb Bluetooth HJX-001 tạo bluetooth cho loa & ...,4.928571,14,33.0,50000.0,10,1,0,0,0,0,0,0,hunglongpc,15431,1064,94,4498,9.661716e+07,4226
15636,15646,Loa 2.0 Kisonli V310,AUDIO,41.0,34000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/0f8b8dd771b74e15dc7f...,https://shopee.vn/Loa-2.0-Kisonli-V310-i.76611...,Loa 2.0 Kisonli V310,4.333333,15,41.0,34000.0,8,12,0,1,0,2,5,3,meocon021184,1257,1634,87,10118,1.401270e+08,986


# Regression Approach

## Extract features

In [9]:
y = full_df['n_sold'].to_numpy() #get the label
y = y
print(y.shape)
print(y.dtype)

(15638,)
float64


In [10]:
#category: categorical
#price: numerical
#shop_address: categorical
#avg_rating: numerical
#n_reviews: numerical
#n_loved: numerical
#shop_n_review: numerical
#shop_n_product: numerical
#shop_rate_feedback: numerical 
#shop_time_feedback: numerical
#shop_age: numerical
#shop_follower: numerical

#Extract features
category = pd.get_dummies(full_df['category'], drop_first=True) #Convert category to one hot encoding
price = full_df[['price']]
shop_address = pd.get_dummies(full_df['shop_address'], drop_first=True)
avg_rating = full_df[['avg_rating']]
n_reviews = full_df[['n_reviews']]
n_loved = full_df[['n_loved']]
shop_n_review = full_df[['shop_n_review']]
shop_n_product = full_df[['shop_n_product']]
shop_rate_feedback = full_df[['shop_rate_feedback']]
shop_time_feedback = full_df[['shop_time_feedback']]
shop_age = full_df[['shop_age']]
shop_follower = full_df[['shop_follower']]


In [11]:
class Metrics():


    @staticmethod
    def computeMSE(y_true, y_pred):
        return np.mean((y_true-y_pred)**2)

    @staticmethod
    def computeMAE(y_true, y_pred):
        return np.mean(np.abs(y_true-y_pred))

    @staticmethod
    def computeSIA(y_true, y_pred, eps=500): #soft interval accuracy
        error = np.abs(y_true - y_pred)
        # print(y_true)
        # print(y_pred)
        # print(error)
        
        # print((error - eps) < 0 )
        res = np.mean(((error - eps) < 0 ) & (y_pred >= 0))
        return res



## Decision Tree Regression

In [12]:
feature_list = [category, 
                price,
                shop_address, 
                avg_rating, 
                n_reviews, 
                n_loved, 
                
                # n_rate_5,
                # n_rate_4,
                # n_rate_3,
                # n_rate_2,
                # n_rate_1,

                shop_n_review, 
                shop_n_product, 
                shop_rate_feedback, 
                shop_time_feedback, 
                shop_age, 
                shop_follower
                ]

In [13]:
X = np.hstack(feature_list)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(10008, 82) (10008,)
(2502, 82) (2502,)
(3128, 82) (3128,)


In [15]:
model = RandomForestRegressor()
model = model.fit(X_train, y_train)


In [17]:
#Compute MAE metrics
mae_train = Metrics.computeMAE(y_train, model.predict(X_train))
mae_val = Metrics.computeMAE(y_val, model.predict(X_val))
mae_test = Metrics.computeMAE(y_test, model.predict(X_test))

print(mae_train)
print(mae_val)
print(mae_test)

158.59727106489413
401.17732427391417
469.23356551932375


In [20]:
#Compute SIA metrics
SIA_train = Metrics.computeSIA(y_train, model.predict(X_train), 1000)
SIA_val = Metrics.computeSIA(y_val, model.predict(X_val), 1000)
SIA_test = Metrics.computeSIA(y_test, model.predict(X_test), 1000)

print(SIA_train)
print(SIA_val)
print(SIA_test)

0.9777178257394085
0.9228617106314948
0.9207161125319693


In [ ]:
model.predict(X_val)[:20]

array([ 923.36,   36.64,  348.92,  253.7 ,  677.8 ,  179.31,  863.45,
        111.5 , 1390.38,  757.14,  641.83, 2055.  , 1699.03,  157.25,
        618.42,  299.17,  164.19, 2167.25,  720.91,  855.57])

In [ ]:
y_val[:20]

array([8.75e+02, 1.00e+00, 9.50e+01, 1.60e+02, 3.20e+03, 1.56e+02,
       5.62e+02, 5.70e+01, 7.11e+02, 6.90e+02, 1.59e+02, 1.70e+03,
       1.10e+03, 3.30e+01, 1.10e+03, 2.83e+02, 1.13e+02, 1.30e+03,
       6.82e+02, 1.50e+03])

## Linear Regression 

In [ ]:
feature_list = [#shop_address, 
                #category, 
                price,

                avg_rating, 
                n_reviews, 
                n_loved, 
                
                # n_rate_5,
                # n_rate_4,
                # n_rate_3,
                # n_rate_2,
                # n_rate_1,

                shop_n_review, 
                shop_n_product, 
                shop_rate_feedback, 
                shop_time_feedback, 
                shop_age, 
                shop_follower
                ]

In [ ]:
X = np.hstack(feature_list)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(10008, 10) (10008,)
(2502, 10) (2502,)
(3128, 10) (3128,)


In [ ]:
#perform feature scaling
normalizer = StandardScaler()

X_train = normalizer.fit_transform(X_train)

In [ ]:
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train ,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
model.coef_

array([ -34.88865325,    5.09426534, 4857.62540335,   28.08422611,
        -22.66189249,  -54.02166846,   14.06205213,   54.47274494,
       -123.1001337 , -335.11862141])

In [ ]:
np.where(model.coef_>1e4)

(array([], dtype=int64),)

In [ ]:
#Compute MAE metrics
mae_train = Metrics.computeMAE(y_train, model.predict(X_train))
mae_val = Metrics.computeMAE(y_val, model.predict(X_val))
mae_test = Metrics.computeMAE(y_test, model.predict(X_test))

print(mae_train)
print(mae_val)
print(mae_test)

600.929456341391
567.0505563574869
612.9852470299271


In [ ]:
#Compute SIA metrics
SIA_train = Metrics.computeSIA(y_train, np.clip(model.predict(X_train), 0,np.inf), 200)
SIA_val = Metrics.computeSIA(y_val, np.clip(model.predict(X_val), 0,np.inf), 200)
SIA_test = Metrics.computeSIA(y_test, np.clip(model.predict(X_test), 0,np.inf), 200)

print(SIA_train)
print(SIA_val)
print(SIA_test)

0.39908073541167066
0.3888888888888889
0.40569053708439895


In [ ]:
#Using L2 Regularization with Grid Search for tuning hyperparameter alpha
from sklearn.linear_model import Ridge

alphas = np.linspace(0.001, 10, num=100)
best_val = np.inf
best_alpha = -1
model = None
for alpha in alphas:
    tmp_model = Ridge(alpha)
    tmp_model.fit(X_train ,y_train)
    mae_val = Metrics.computeMAE(y_val, tmp_model.predict(X_val))
    if mae_val < best_val:
        best_val = mae_val
        model = tmp_model
        best_alpha = alpha

print("Best cross-validation MAE: {} with alpha: {}".format(best_val, best_alpha))

Best cross-validation MAE: 582.7109424754536 with alpha: 10.0


In [ ]:
#Compute MAE metrics
mae_train = Metrics.computeMAE(y_train, model.predict(X_train))
mae_val = Metrics.computeMAE(y_val, model.predict(X_val))
mae_test = Metrics.computeMAE(y_test, model.predict(X_test))

print(mae_train)
print(mae_val)
print(mae_test)

609.2475757851719
582.7109424754536
615.9540970026771


In [ ]:
print(model.coef_)

[ 1.82288609e+01 -1.68417490e+01 -1.23192899e+00  6.52743316e-02
 -1.11658619e+01 -7.03823417e+00  1.14088497e+01 -6.56725935e-02
 -6.75363349e+00 -6.02116033e+00  0.00000000e+00 -4.52856556e+00
 -5.81350353e+00 -1.37244448e+01  8.77068719e+00 -4.71180187e+00
  1.68000542e+02  0.00000000e+00  2.25323115e+00  1.97097979e+01
 -6.60426603e+00  1.21152996e+01 -6.92175441e+00  2.59520404e+00
  0.00000000e+00  7.62051155e-01 -7.50530106e+00 -3.38299723e+00
  3.99865518e+00 -1.47742903e+01  1.76738989e+01  1.71631875e+01
  6.87664043e+00  3.89311242e+00 -1.36693534e+02  8.22636244e+01
  4.46067024e+00 -1.31372951e+01  1.38378442e+01  3.35915748e+00
 -1.25702835e+00  3.22046675e+01  3.39359953e+00 -5.05199121e+00
  4.35872174e+01  8.31312264e+00 -5.20375349e+00  0.00000000e+00
 -2.86973195e+01  0.00000000e+00  9.05432478e-01  1.38335853e+00
 -1.28267571e+01  6.81512882e+00 -4.11008808e+01  7.62247684e+01
 -6.92057288e+00  5.26026131e+02  1.72028457e+02  1.63563192e+02
 -1.64392232e+01 -2.40655

In [ ]:
#Compute SIA metrics
SIA_train = Metrics.computeSIA(y_train, np.clip(model.predict(X_train), 0,np.inf), 200)
SIA_val = Metrics.computeSIA(y_val, np.clip(model.predict(X_val), 0,np.inf), 200)
SIA_test = Metrics.computeSIA(y_test, np.clip(model.predict(X_test), 0,np.inf), 200)

print(SIA_train)
print(SIA_val)
print(SIA_test)

0.47302158273381295
0.45323741007194246
0.4670716112531969


### Linear regression with Mean Encoding

In [ ]:
df_train, df_test = train_test_split(full_df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=1)
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(10008, 28)
(2502, 28)
(3128, 28)


In [ ]:
#Get a lookup dict for each categorical features
#Dict.key is the value of categorical variable, and Dict.value is the mean encoding of that value
shop_address_dict = df_train.groupby("shop_address").agg(["mean"])["n_sold"].to_dict()['mean']
category_dict = df_train.groupby("category").agg(["mean"])["n_sold"].to_dict()['mean']
category_dict

{'AUDIO': 169.35285505124452,
 'CAPSAC': 2595.850381679389,
 'CHUOTBANPHIM': 857.9598145285935,
 'DIENTHOAI': 800.1085899513777,
 'GAYCHUPHINH': 201.48318042813455,
 'LAPTOP': 70.70522979397781,
 'LINHKIENMAYTINH': 475.09400324149107,
 'MAYBAN': 45.858034321372855,
 'MAYTINHBANG': 61.790286975717436,
 'MIENGDAN': 4068.2828438948995,
 'OCUNG': 448.2113402061856,
 'PHUKIEN': 518.781990521327,
 'PINSAC': 973.0367278797996,
 'SIM': 1471.7198142414861,
 'THIETBIMANG': 468.15179968701096,
 'VOBAOOPLUNG': 5299.911144578313}

In [ ]:
#category: categorical
#price: numerical
#shop_address: categorical
#avg_rating: numerical
#n_reviews: numerical
#n_loved: numerical
#shop_n_review: numerical
#shop_n_product: numerical
#shop_rate_feedback: numerical 
#shop_time_feedback: numerical
#shop_age: numerical
#shop_follower: numerical

#Extract features

price = full_df[['price']]

avg_rating = full_df[['avg_rating']]
n_reviews = full_df[['n_reviews']]
n_loved = full_df[['n_loved']]
shop_n_review = full_df[['shop_n_review']]
shop_n_product = full_df[['shop_n_product']]
shop_rate_feedback = full_df[['shop_rate_feedback']]
shop_time_feedback = full_df[['shop_time_feedback']]
shop_age = full_df[['shop_age']]
shop_follower = full_df[['shop_follower']]


In [ ]:
category = full_df['category'].map(category_dict).fillna(0)
category.unique()

array([ 800.10858995,   61.79028698, 5299.91114458, 4068.28284389,
        973.03672788, 2595.85038168,  201.48318043, 1471.71981424,
         70.70522979,   45.85803432,  475.09400324,  857.95981453,
        468.15179969,  448.21134021,  518.78199052,  169.35285505])

In [ ]:
shop_address = full_df['shop_address'].map(shop_address_dict).fillna(0)
shop_address.unique()

array([ 682.25696685,  134.2       , 1802.88811815,  723.81      ,
        250.32786885,  234.30952381,  311.82608696, 1859.675     ,
       1424.75      ,  795.22222222,  714.66666667,  244.58333333,
        716.03571429,  472.86046512,  561.80769231,  562.34482759,
        157.        ,  452.72131148,  858.02564103,  409.33333333,
        448.23529412, 1240.375     ,   53.6       ,  862.73000941,
        720.31111111,  110.14285714,   25.16666667,   70.25      ,
       2480.34375   ,    0.        ,  115.        ,  774.53846154,
        166.        ,  447.825     ,  910.        ,  148.73333333,
         94.        ,  158.        ,  704.46153846,  389.63157895,
        286.31428571,  568.        ,   82.5       ,  279.3       ,
        140.        ,   67.25      ,  153.        ,   30.        ,
        321.5       ,   31.42857143,   13.33333333,    4.        ,
        313.25      ,  244.4       ])

In [ ]:
feature_list = [shop_address[:,None], 
                category[:,None], 
                price,

                avg_rating, 
                n_reviews, 
                n_loved, 

                shop_n_review, 
                shop_n_product, 
                shop_rate_feedback, 
                shop_time_feedback, 
                shop_age, 
                shop_follower
                ]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  


In [ ]:
X = np.hstack(feature_list)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(10008, 12) (10008,)
(2502, 12) (2502,)
(3128, 12) (3128,)


In [ ]:
y_train == df_train['n_sold'].to_numpy()

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
#perform feature scaling
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train ,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
model.coef_

array([ 7.38991498e+01,  1.07909964e+02, -1.71420666e+01, -4.73916159e-01,
        4.84214899e+03,  2.23704893e+01, -1.59027196e+02, -4.87365074e+01,
       -3.70200110e+00,  4.16535493e+01, -1.11751193e+02, -2.61049457e+02])

In [ ]:
#Compute MAE metrics
mae_train = Metrics.computeMAE(y_train, model.predict(X_train))
mae_val = Metrics.computeMAE(y_val, model.predict(X_val))
mae_test = Metrics.computeMAE(y_test, model.predict(X_test))

print(mae_train)
print(mae_val)
print(mae_test)

602.964315609281
565.97680477335
611.1476004126301


In [ ]:
#Compute SIA metrics
SIA_train = Metrics.computeSIA(y_train, np.clip(model.predict(X_train), 0,np.inf), 200)
SIA_val = Metrics.computeSIA(y_val, np.clip(model.predict(X_val), 0,np.inf), 200)
SIA_test = Metrics.computeSIA(y_test, np.clip(model.predict(X_test), 0,np.inf), 200)

print(SIA_train)
print(SIA_val)
print(SIA_test)

0.43445243804956035
0.4248601119104716
0.42998721227621484


## Neural Network Regression

In [ ]:
df_train, df_test = train_test_split(full_df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=1)
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(10008, 28)
(2502, 28)
(3128, 28)


In [ ]:
#Get a lookup dict for each categorical features
#Dict.key is the value of categorical variable, and Dict.value is the mean encoding of that value
shop_address_dict = df_train.groupby("shop_address").agg(["mean"])["n_sold"].to_dict()['mean']
category_dict = df_train.groupby("category").agg(["mean"])["n_sold"].to_dict()['mean']
#mean encoding
category = full_df['category'].map(category_dict).fillna(0)[:,None] #Fill values that do not exist in train set by zero
shop_address = full_df['shop_address'].map(shop_address_dict).fillna(0)[:,None]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  import sys


In [ ]:
#onehot encoding
category = pd.get_dummies(full_df['category'], drop_first=True) #Convert category to one hot encoding
shop_address = pd.get_dummies(full_df['shop_address'], drop_first=True)

In [ ]:
feature_list = [shop_address, 
                category, 
                price,

                avg_rating, 
                n_reviews, 
                n_loved, 

                shop_n_review, 
                shop_n_product, 
                shop_rate_feedback, 
                shop_time_feedback, 
                shop_age, 
                shop_follower
                ]

In [ ]:
X = np.hstack(feature_list)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(10008, 82) (10008,)
(2502, 82) (2502,)
(3128, 82) (3128,)


In [ ]:
#perform feature scaling
normalizer = StandardScaler()

X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

In [ ]:
# from sklearn.neural_network import MLPRegressor
# batch_size = 200
# model = MLPRegressor(hidden_layer_sizes=[64], activation='relu',
#                      verbose=True, 
#                      max_iter=1000, 
#                      batch_size=batch_size, n_iter_no_change=10000,
#                      learning_rate='adaptive',
#                      alpha=0.0,
#                      random_state=1,warm_start=True)
# model.out_activation_ ='relu'
# model.fit(X_train, y_train)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=32,activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dense(units=64, activation ='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dense(units=1, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss = tf.keras.losses.mean_absolute_error,
              metrics=['mae'])


            

In [ ]:
if not os.path.exists("models"):
    os.makedirs("models")
mcp_save = tf.keras.callbacks.ModelCheckpoint(
    filepath="models/sold_prediction_nn_model.h5",
    save_weights_only=True,
    monitor='val_mae',
    mode='min',
    save_best_only=True)  #save model's weights with highest validation performance

model.fit(X_train, y_train,
          epochs=300,
          batch_size=128,
          validation_data=(X_val, y_val),
          callbacks=[mcp_save])

Epoch 1/300
79/79 [==============================] - 3s 7ms/step - loss: 1203.1428 - mae: 1202.2969 - val_loss: 993.8815 - val_mae: 992.8731
Epoch 2/300
79/79 [==============================] - 0s 3ms/step - loss: 1172.5590 - mae: 1171.3307 - val_loss: 937.6791 - val_mae: 935.4615
Epoch 3/300
79/79 [==============================] - 0s 3ms/step - loss: 1102.7432 - mae: 1100.0798 - val_loss: 854.4343 - val_mae: 850.4847
Epoch 4/300
79/79 [==============================] - 0s 3ms/step - loss: 1025.2854 - mae: 1021.0096 - val_loss: 799.8625 - val_mae: 794.9591
Epoch 5/300
79/79 [==============================] - 0s 3ms/step - loss: 857.4585 - mae: 852.4780 - val_loss: 767.4009 - val_mae: 762.0483
Epoch 6/300
79/79 [==============================] - 0s 3ms/step - loss: 936.8790 - mae: 931.4238 - val_loss: 747.6791 - val_mae: 742.2338
Epoch 7/300
79/79 [==============================] - 0s 3ms/step - loss: 998.9962 - mae: 993.5033 - val_loss: 735.5134 - val_mae: 729.9274
Epoch 8/300
79/79 [

In [ ]:
model.load_weights("models/sold_prediction_nn_model.h5")

In [ ]:
#Compute MAE metrics
mae_train = Metrics.computeMAE(y_train, model.predict(X_train).squeeze())
mae_val = Metrics.computeMAE(y_val, model.predict(X_val).squeeze())
mae_test = Metrics.computeMAE(y_test, model.predict(X_test).squeeze())

print(mae_train)
print(mae_val)
print(mae_test)

403.99230465138083
387.9702935638092
428.46827630959734


In [ ]:
#Compute SIA metrics
SIA_train = Metrics.computeSIA(y_train, np.clip(model.predict(X_train), 0,np.inf).squeeze(), 200)
SIA_val = Metrics.computeSIA(y_val, np.clip(model.predict(X_val).squeeze(), 0,np.inf), 200)
SIA_test = Metrics.computeSIA(y_test, np.clip(model.predict(X_test).squeeze(), 0,np.inf), 200)

print(SIA_train)
print(SIA_val)
print(SIA_test)

0.7255195843325339
0.6962430055955235
0.6911764705882353
